In [1]:
import os
import gc
from joblib import dump, load
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# load files

In [2]:
train = pd.read_parquet("../input/train_full_features.parquet")

# define loss and metric

In [3]:
def amex_metric(y_true, y_pred):
    
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    
    for i in [1, 0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
        
    return 0.5 * (gini[1]/gini[0] + top_four)

In [4]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return "amex_metric", amex_metric(y_true, y_pred), True

# define training config

In [5]:
seed = 42
n_folds = 5

features = load("selected_features.pkl")

target = "target"

cat_features_base = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
] 
cat_features = [
    "{}_last".format(feature) for feature in cat_features_base
]
cat_features = [feature for feature in cat_features if feature in features]
            
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting": "dart",
    "seed": seed,
    "num_leaves": 100,
    "learning_rate": 0.01,
    "feature_fraction": 0.20,
    "bagging_freq": 10,
    "bagging_fraction": 0.50,
    "n_jobs": -1,
    "lambda_l2": 2,
    "min_data_in_leaf": 40,
}

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything(seed)

In [6]:
target_fold = 4

kfold = StratifiedKFold(
    n_splits=n_folds, 
    shuffle=True, 
    random_state=seed
)

for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[target])):
    
    if fold == target_fold:
        break

x_train, x_val = train.loc[trn_ind, features], train.loc[val_ind, features]
y_train, y_val = train.loc[trn_ind, target], train.loc[val_ind, target]

lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)

In [7]:
del train, x_train, x_val, y_train, y_val
gc.collect()

0

In [8]:
save_folder = os.path.join("../ckpt/lgbm_seed_{}".format(seed))
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
save_path = os.path.join(save_folder, "fold_{}".format(target_fold))
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [9]:
print("#" * 50)
print("Training fold {} with {} features...".format(target_fold, len(features)))

global max_score 
max_score = 0.793

def save_model():
    def callback(env):
        global max_score
        iteration = env.iteration
        score = env.evaluation_result_list[3][2]
        if iteration % 100 == 0:
            print("iteration {}, score= {:.05f}".format(iteration,score))
        if score > max_score:
            max_score = score
            print("High Score: iteration {}, score={:.05f}".format(iteration, score))
            dump(env.model, os.path.join(save_path, "{:.05f}.pkl".format(score)))

    callback.order = 0
    return callback

model = lgb.train(
    params=params,
    train_set=lgb_train,
    num_boost_round=20000,
    valid_sets = [lgb_train, lgb_valid],
    early_stopping_rounds=100,
    verbose_eval = 500,
    feval = lgb_amex_metric,
    callbacks=[save_model()],
)

##################################################
Training fold 4 with 1950 features...


/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 95063, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.201701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 211763
[LightGBM] [Info] Number of data points in the train set: 367131, number of used features: 1950


/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258935 -> initscore=-1.051512
[LightGBM] [Info] Start training from score -1.051512
iteration 0, score= 0.68819


/home/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


iteration 100, score= 0.76033
iteration 200, score= 0.76238
iteration 300, score= 0.76329
iteration 400, score= 0.76529
[500]	training's binary_logloss: 0.337611	training's amex_metric: 0.777464	valid_1's binary_logloss: 0.339416	valid_1's amex_metric: 0.768159
iteration 500, score= 0.76811
iteration 600, score= 0.77025
iteration 700, score= 0.77244
iteration 800, score= 0.77405
iteration 900, score= 0.77648
[1000]	training's binary_logloss: 0.246655	training's amex_metric: 0.794357	valid_1's binary_logloss: 0.251986	valid_1's amex_metric: 0.778221
iteration 1000, score= 0.77822
iteration 1100, score= 0.77990
iteration 1200, score= 0.78129
iteration 1300, score= 0.78295
iteration 1400, score= 0.78445
[1500]	training's binary_logloss: 0.222494	training's amex_metric: 0.807433	valid_1's binary_logloss: 0.231772	valid_1's amex_metric: 0.785057
iteration 1500, score= 0.78495
iteration 1600, score= 0.78727
iteration 1700, score= 0.78813
iteration 1800, score= 0.78876
iteration 1900, score= 

iteration 5000, score= 0.79772
High Score: iteration 5007, score=0.79786
High Score: iteration 5027, score=0.79791
High Score: iteration 5028, score=0.79791
iteration 5100, score= 0.79759
High Score: iteration 5123, score=0.79793
High Score: iteration 5138, score=0.79793
High Score: iteration 5139, score=0.79793
High Score: iteration 5140, score=0.79793
High Score: iteration 5143, score=0.79804
iteration 5200, score= 0.79779
iteration 5300, score= 0.79775
iteration 5400, score= 0.79775
[5500]	training's binary_logloss: 0.167621	training's amex_metric: 0.885906	valid_1's binary_logloss: 0.21391	valid_1's amex_metric: 0.797736
iteration 5500, score= 0.79772
High Score: iteration 5553, score=0.79806
High Score: iteration 5566, score=0.79806
iteration 5600, score= 0.79802
High Score: iteration 5606, score=0.79807
High Score: iteration 5610, score=0.79815
High Score: iteration 5611, score=0.79815
High Score: iteration 5612, score=0.79817
High Score: iteration 5655, score=0.79818
High Score:

iteration 14000, score= 0.79967
iteration 14100, score= 0.79941
iteration 14200, score= 0.79949
iteration 14300, score= 0.79920
iteration 14400, score= 0.79922
[14500]	training's binary_logloss: 0.104051	training's amex_metric: 0.979689	valid_1's binary_logloss: 0.212783	valid_1's amex_metric: 0.799582
iteration 14500, score= 0.79964
iteration 14600, score= 0.79922
iteration 14700, score= 0.79942
iteration 14800, score= 0.79981
iteration 14900, score= 0.79965
[15000]	training's binary_logloss: 0.101692	training's amex_metric: 0.98206	valid_1's binary_logloss: 0.21283	valid_1's amex_metric: 0.799248
iteration 15000, score= 0.79927
iteration 15100, score= 0.79923
iteration 15200, score= 0.79917
iteration 15300, score= 0.79904
iteration 15400, score= 0.79889
[15500]	training's binary_logloss: 0.099218	training's amex_metric: 0.984346	valid_1's binary_logloss: 0.212929	valid_1's amex_metric: 0.799093
iteration 15500, score= 0.79905
iteration 15600, score= 0.79922
iteration 15700, score= 0.

In [10]:
# def training(train):
    
#     # create a numpy array to store out of folds predictions
#     oof_predictions = np.zeros(len(train))
    
#     kfold = StratifiedKFold(
#         n_splits=n_folds, 
#         shuffle=True, 
#         random_state=seed
#     )
    
#     for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[target])):
        
#         print("#" * 50)
#         print("Training fold {} with {} features...".format(fold, len(features)))
        
#         x_train, x_val = train.loc[trn_ind, features], train.loc[val_ind, features]
#         y_train, y_val = train.loc[trn_ind, target], train.loc[val_ind, target]
        
#         lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
#         lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)
#         model = lgb.train(
#             params=params,
#             train_set=lgb_train,
#             num_boost_round=10500,
#             valid_sets = [lgb_train, lgb_valid],
#             early_stopping_rounds=100,
#             verbose_eval = 500,
#             feval = lgb_amex_metric
#         )
#         # save best model
#         dump(model, "../ckpt/lgbm_{}_{}.pkl".format(fold, seed))
        
#         # predict validation
#         val_pred = model.predict(x_val)
        
#         # add to out of folds array
#         oof_predictions[val_ind] = val_pred
        
#         # compute fold metric
#         score = amex_metric(y_val, val_pred)
#         print("fold {} score is {}".format(fold, score))
        
#         del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
#         gc.collect()
        
#     # compute oof
#     score = amex_metric(train[target], oof_predictions)
#     print("oof score is {}".format(score))
    
#     # create a dataframe to store out of folds predictions
#     oof_df = pd.DataFrame({"customer_ID": train["customer_ID"], "target": train[target], "prediction": oof_predictions})
#     oof_df.to_parquet("lgbm_oof_{}.parquet".format(seed))
    
#     return oof_df